Znalazłem bibliotekę w Pythonie od tego https://github.com/pyusb/pyusb. 
Potem jakieś problemy z backendem, dużo wywalało "no backend available".
Znalazłem libusb https://libusb.info/ libusb is a C library that provides generic access to USB devices. Wziąłem dlla, dodałem do zmiennych środowiskowych i zaczęło działać.

Korzystając z dokumentacji od pyusb, znalazłem przydatne funkcje

In [ ]:
import usb.core

print(usb.core.show_devices())

for device in usb.core.find(find_all=True):
    print(device)

Pokaż urządzenia z podłączonym missile launcherem i bez, porównaj - vendor ID i model ID znane.

In [ ]:
dev = usb.core.find(idVendor=0x0a81, idProduct=0x0701)
if dev is None:
    raise ValueError('Device not found')
print(dev)

Jedna konfiguracja, jeden interfejs, jeden endpoint. Zdziwiłem się, że tylko jeden endpoint do odsłuchiwania, ale jak próbowałem coś z niego wyciągnąć to nie wypluwał żadnego info.

Chatgpt podpowiedział, ale wszystko jest w dokumentacji usb i tutaj https://www.beyondlogic.org/usbnutshell/usb6.shtml#StandardInterfaceRequests:
Control Transfer Parameters:
bmRequestType: 0x21 (Host to Device, Class, Interface)
bRequest: 0x09 (This is often the "Set_Report" request for HID devices, used to send data)
wValue: 0x0200 (Indicates that you're sending an output report)
wIndex: Interface number, usually 0 for simple devices
data: The data you want to send (in this case, a simple 1-byte array [0x01])

In [ ]:
data = [0x01]
dev.ctrl_transfer(0x21, 0x09, 0x0200, 0, data)

Ok, pierwszy ruch. Potem wartości od 0 do 255 puszczałem i patrzyłem co się dzieje, mniej więcej tym kodem, ale nie pamiętam do końca

In [3]:
import usb.core
import usb.util
import keyboard  # Import the keyboard library
import time

# Find the device
dev = usb.core.find(idVendor=0x0a81, idProduct=0x0701)

if dev is None:
    raise ValueError('Device not found')

# Set configuration (assuming there's only one configuration)
dev.set_configuration()

# Initialize the value to send
value = 0

print("Press 'shift' to send data. Press 'Esc' to exit.")

# Loop to listen for keyboard events
while True:
    # Wait for 'shift' key press
    if keyboard.is_pressed('shift'):

        # Prepare the data to send (single byte)
        data = [value]  # Create a list with the current byte value

        try:
            # Control transfer to send data
            dev.ctrl_transfer(0x21, 0x09, 0x0200, 0, data)
            print(f"Sent data: {hex(value)}")  # Print the value sent in hex

            # Increment the value, reset to 0 if it exceeds 255
            value = (value + 1) % 256

            # Wait a moment to prevent multiple sends from a single key press
            #keyboard.wait('shift', suppress=True)  # Wait for release of 'shift'

        except usb.core.USBError as e:
            print(f"Error sending data {hex(value)}: {e}")

    # Allow user to exit with 'Esc' key
    elif keyboard.is_pressed('esc'):
        print("Exiting...")
        break
    time.sleep(0.1)

Press 'Enter' to send data. Press 'Esc' to exit.
Sent data: 0x0
Sent data: 0x1
Exiting...


0x4 lewo 
0x8 prawo
0x2 góra 
0x1 dół
0x10 strzał
0x20 stop
Wszystko miałem, jakieś proste GUI i działa.